In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re
import random ##kann raus
from datetime import datetime
from collections import OrderedDict


def extract_pattern(string, pattern):
    match = re.search(pattern, string)
    if match:
        return match.group(1)
    else:
        return None

def format_timestring(timestring):
    timestring = timestring.replace('a.m.','AM').replace('p.m.','PM')
    b= timestring.split()
    if len(b[0])>3:
            b[0] = b[0][:3]+'.'
    if len(b[3]) not in [4,5]:
            b[3] = '0'+b[3]+':00'
    b=' '.join(b)
    return b
#ThoraxCBCT:     2c0360d0-ee97-4b37-a950-9ad1edb1ed5b
initial_uid = '2c0360d0-ee97-4b37-a950-9ad1edb1ed5b'



### Fetch Results

In [2]:
###ThoraxCBCT
url_leaderboard = 'https://learn2reg.grand-challenge.org/evaluation/thoraxcbct/leaderboard/' ##ThroaxCBCT 
max_submissions = 1000
#submissions = {}
#ordered dict
submissions = OrderedDict()

##regex patterns
username_pattern = r'<a\s+href="[^"]+\/users\/([^\/]+)\/"[^>]*>[^<]+<\/a>'
team_pattern = r'\(<a\s+href="[^"]+">(.*?)<\/a>\)'
uid_pattern = r'\/evaluation\/([\w-]+)\/'

url_suffix = '?draw=1&columns[0][data]=0&columns[0][name]=&columns[0][searchable]=true&columns[0][orderable]=true&columns[0][search][value]=&columns[0][search][regex]=false&order[0][column]=0&order[0][dir]=asc&start=0&length={}&search[value]=&search[regex]=false&_=1686904037262'.format(max_submissions)
response = requests.get(url_leaderboard+url_suffix, headers={'X-Requested-With': 'XMLHttpRequest'})
num_records = int(response.json()['recordsTotal'])
table_data = response.json()['data']
print('Found {} entries'.format(num_records))

for ii,examplary_entry in enumerate(range(len(table_data))):
        #probably better with regex
        submission_uid = extract_pattern(table_data[examplary_entry][3], uid_pattern)
        submission_user = extract_pattern(table_data[examplary_entry][1], username_pattern)
        submission_team = extract_pattern(table_data[examplary_entry][1], team_pattern)
        submissions[submission_uid] = {'user': submission_user, 'team': submission_team}

        ##get json directly
        url_submission = 'https://learn2reg.grand-challenge.org/evaluation/'+submission_uid+'/'
        response_submission = requests.get(url_submission)
        soup = BeautifulSoup(response_submission.text, 'html.parser')
        results = json.loads(str(soup.findAll('pre')[0])[5:-6])
        #time
        time_string = soup.findAll('dt')[7].next_sibling.next_sibling.text
        time = datetime.strptime(format_timestring(time_string), '%b. %d, %Y, %I:%M %p')
        
        submissions[submission_uid] = {'user': submission_user, 'team': submission_team, 'table_rank':str(ii+1), 'time':str(time), 'results': results}
        print(ii,submission_uid, submission_user, submission_team, time, sep='\t')
        
print('Fetched {} entries'.format(len(submissions)))

##save dict as sorted json
#with open('L2R2023_Snapshot_ThoraxCBCT.json', 'w') as fp:
#    json.dump(submissions, fp, indent=4, sort_keys=False)
    


Found 40 entries
0	8facfcdd-ebd4-4dcc-8dea-a5ac28a300b3	SSKJLBW	ikun	2023-08-06 18:47:00
1	4e7cd10e-fe13-4da6-ad12-fb1d018304db	alisonbrielee	None	2023-08-03 05:23:00
2	0170280d-e4f1-428d-bc18-c50b084cd76e	alisonbrielee	None	2023-07-18 02:53:00
3	d23610fc-a2cd-4a23-87cc-f2f1d11ee392	alisonbrielee	None	2023-07-14 14:18:00
4	fb6600ad-2788-4c31-a692-43ce500add9b	SSKJLBW	ikun	2023-08-06 14:51:00
5	a94579e8-488d-4263-929e-f5d9a51f2775	SSKJLBW	ikun	2023-08-05 13:50:00
6	3adf9270-0d32-4223-b4a3-1c24e5b2cf3e	alisonbrielee	None	2023-07-20 06:22:00
7	2f2401c9-face-40f3-8acf-bb85dc0f9ff6	SSKJLBW	ikun	2023-07-31 11:08:00
8	36f7ac29-8bb4-48c6-bd2c-40136ce6df7e	SSKJLBW	ikun	2023-07-28 14:00:00
9	d10e8bab-cf06-400a-a5ac-7f6dd71a5345	SSKJLBW	ikun	2023-08-03 13:55:00
10	9ad56d63-65f7-4318-922a-ac09536afeb2	alisonbrielee	None	2023-08-04 05:31:00
11	0913df04-7e3e-461c-abec-68fc2bfc8444	chrgro	Challenge Organizers	2023-07-12 14:49:00
12	19488cce-7a4e-4060-b4fb-b5fa05f769e3	wiebkeheyer	Challenge Organizers

In [3]:

beginningSubmission = datetime.strptime('2023-07-08','%Y-%m-%d')
endSubmission = datetime.strptime('2023-08-06 23:59:59','%Y-%m-%d %H:%M:%S')


#with open('L2R2023_Snapshot_ThroaxCBCT.json') as json_file:
#    submissions = json.load(json_file)
#
#results = OrderedDict()
#for submission in results_json:
#    submission_date = datetime.strptime(results_json[submission]['time'],'%Y-%m-%d %H:%M:%S')
#    if submission_date > beginningSubmission and submission_date < endSubmission:
#        results[submission] = results_json[submission]


##add initial as well
#ThoraxCBCT:     2c0360d0-ee97-4b37-a950-9ad1edb1ed5b
#initial_uid = '2c0360d0-ee97-4b37-a950-9ad1edb1ed5b'
#results[initial_uid] = results_json[initial_uid]
#print('Before:' + str(len(results_json)), 'After:' + str(len(results)))





##Remove submissions that are outside of the submission window
results = OrderedDict()
for submission in submissions:
    submission_date = datetime.strptime(submissions[submission]['time'],'%Y-%m-%d %H:%M:%S')
    if submission_date > beginningSubmission and submission_date < endSubmission:
        results[submission] = submissions[submission]
print('Before:' + str(len(submissions)), 'After:' + str(len(results)))

results[initial_uid] = submissions[initial_uid]
results[initial_uid]['user']='INITIAL'
results[initial_uid]['team']=None






Before:40 After:40


### Calculate Ranks

In [4]:
import numpy as np
import scipy.stats as stats
import scipy 
def unique(list1):
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    return unique_list
np.set_printoptions(precision=3)


In [5]:
sample_submission = results[list(results.keys())[0]]
all_cases = list(sample_submission['results']['cases'])


#######
uids = list(results.keys())
users = [results[k]['user'] for k in results.keys()]
teams = [results[k]['team'] for k in results.keys()]
table_rank = [results[k]['table_rank'] for k in results.keys()]
results_q = [results[k]['results'] for k in results.keys()]



DSC = np.empty((len(users),len(all_cases), 15))
HD95 = np.empty((len(users),len(all_cases), 15))
TRE_kp = np.empty((len(users), 121380))

LogJacDetStd = np.ones((len(users),len(all_cases)))

for i, uid in enumerate(uids):
    tmp_kp = []
    for j, case in enumerate(all_cases):
        DSC[i,j,:] = results_q[i]['cases'][case]['DSC']['detailed']
        HD95[i,j,:] = results_q[i]['cases'][case]['HD95']['detailed']
        LogJacDetStd[i,j] = results_q[i]['cases'][case]['LogJacDetStd']['detailed']
        tmp_kp += results_q[i]['cases'][case]['TRE_kp']['detailed']
    TRE_kp[i] = tmp_kp

DSC = np.nan_to_num(DSC)
HD95 = np.nan_to_num(HD95)

N = len(uids)

##zero displacements
idx0 = uids.index(initial_uid)
LogJacDetStd[idx0] = 5.0 ##set large value for initial 


###robistyfy
DSC_30 = np.zeros((N, round((len(all_cases)*15)*.3)))
idx30 = np.argsort(DSC[idx0].reshape(-1))[:DSC_30.shape[1]]
for i in range(N):
    DSC_30[i] = DSC[i].reshape(-1)[idx30]







In [6]:
import scipy
import scipy.stats
p_threshold = 0.05
def scores_better(task_metric):
    better = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            h,p = scipy.stats.ranksums(task_metric[i], task_metric[j])
            if((h>0)&(p<p_threshold)): #sign of h and p-value
                better[i,j] = 1
    scores_task = better.sum(0)
    return scores_task

def rankscore_avgtie(scores_int):
    N = len(scores_int)
    rankscale = np.linspace(.1,1,N) #our definition
    rankavg = np.zeros((N,2))
    scorerank = np.zeros(N)
    #argsort with reverse index
    idx_ = np.argsort(scores_int)
    idx = np.zeros(N).astype('int32')
    idx[idx_] = np.arange(N)
    #averaging ties
    for i in range(N):
        rankavg[scores_int[i],0] += rankscale[idx[i]]
        rankavg[scores_int[i],1] += 1
    rankavg = rankavg[:,0]/np.maximum(rankavg[:,1],1e-6)
    for i in range(N):
        scorerank[i] = rankavg[scores_int[i]]
    return scorerank
scores = np.array([0, 2, 4, 3, 4, 0]).astype('int32')
scorerank = rankscore_avgtie(scores)

def greaters(scores):
    return np.sum(scores.reshape(1,-1)>scores.reshape(-1,1),0)


In [7]:
rank_all = np.zeros((N,5))
#TASK3
subset = DSC.reshape(N,-1)
scores = scores_better(subset)
rank_dice3a = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,0] = rank_dice3a

subset = DSC_30.reshape(N,-1)
scores = scores_better(subset)
rank_dice3b = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,1] = rank_dice3b

subset = HD95.reshape(N,-1)
scores = scores_better(-subset)
rank_hd3 = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,2] = rank_hd3

subset = LogJacDetStd.reshape(N,-1)
scores = scores_better(-subset)
rank_jac3 = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,3] = rank_jac3


subset = LogJacDetStd.reshape(N,-1)
scores = scores_better(-subset)
rank_tre = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,4] = rank_tre

In [8]:
DSC_mean = DSC.reshape(N,-1).mean(1)
DSC_30_mean = DSC_30.reshape(N,-1).mean(1)
LogJacDetStd_mean = LogJacDetStd.reshape(N,-1).mean(1)
HD95_mean = HD95.reshape(N,-1).mean(1)
TRE_kp_mean = TRE_kp.reshape(N,-1).mean(1)


In [9]:
rank3 = np.power(np.prod(rank_all[:,:],axis=1),.2)
all_rank3 = np.power(np.prod(rank_all[:,:],axis=1),1/5)
idx2 = np.argsort(-all_rank3)



In [10]:
idx2 = np.argsort(-all_rank3)
for i in range(N):
    team = users[idx2[i]].replace("0","O").zfill(25).replace("0", " ")
    #print(i+1,'\t',team,'\t','%0.3f'%all_rank[idx2[i]],'\t','%0.3f'%TRE_lm_mean[idx2[i]],'\t','%0.3f'%np.mean(LogJacDetStd,1)[idx2[i]],'\t','https://learn2reg.grand-challenge.org/evaluation/'+uids[idx2[i]]+'/')
    gc_team = teams[idx2[i]] if str(teams[idx2[i]]) != 'None' else ' '
    print(i+1,'\t',team,'\t',gc_team,3*'\t','%0.3f'%all_rank3[idx2[i]],'\t','%0.3f'%DSC_mean[idx2[i]],'\t','%0.3f'%LogJacDetStd_mean[idx2[i]],'\t','https://learn2reg.grand-challenge.org/evaluation/'+uids[idx2[i]]+'/')


1 	             alisonbrielee 	   			 0.801 	 0.537 	 0.038 	 https://learn2reg.grand-challenge.org/evaluation/3adf9270-0d32-4223-b4a3-1c24e5b2cf3e/
2 	                   SSKJLBW 	 ikun 			 0.763 	 0.505 	 0.080 	 https://learn2reg.grand-challenge.org/evaluation/042fe8db-27c0-4ec2-9495-0ca563baa391/
3 	                   SSKJLBW 	 ikun 			 0.757 	 0.534 	 0.073 	 https://learn2reg.grand-challenge.org/evaluation/2f2401c9-face-40f3-8acf-bb85dc0f9ff6/
4 	                   SSKJLBW 	 ikun 			 0.751 	 0.551 	 0.108 	 https://learn2reg.grand-challenge.org/evaluation/a94579e8-488d-4263-929e-f5d9a51f2775/
5 	               wiebkeheyer 	 Challenge Organizers 			 0.748 	 0.504 	 0.063 	 https://learn2reg.grand-challenge.org/evaluation/19488cce-7a4e-4060-b4fb-b5fa05f769e3/
6 	                   SSKJLBW 	 ikun 			 0.732 	 0.530 	 0.109 	 https://learn2reg.grand-challenge.org/evaluation/36f7ac29-8bb4-48c6-bd2c-40136ce6df7e/
7 	             alisonbrielee 	   			 0.729 	 0.501 	 0.112 	 https://learn

In [11]:
##Remove team and participant duplicates

tmp_teams=[]
tmp_users=[]
idx3 = idx2.tolist().copy()
for i in idx2:
    if (teams[i] in tmp_teams and teams[i] != None) or (users[i] in tmp_users):
        idx3.remove(i)
    else:
        tmp_teams.append(teams[i])
        tmp_users.append(users[i])


final_idx = np.array(idx3)

for ii,idx in enumerate(final_idx):
    team = users[idx].replace("0","O").zfill(25).replace("0", " ")
    gc_team = teams[idx] if str(teams[idx]) != 'None' else ' '
    print(ii+1,'\t',team,'\t',gc_team,3*'\t','%0.3f'%all_rank3[idx],'\t','%0.3f'%DSC_mean[idx],'\t','%0.3f'%LogJacDetStd_mean[idx],'\t','https://learn2reg.grand-challenge.org/evaluation/'+uids[idx2[i]]+'/')


1 	             alisonbrielee 	   			 0.801 	 0.537 	 0.038 	 https://learn2reg.grand-challenge.org/evaluation/f8ca0216-f5ac-4ad4-8899-c08bf642cf49/
2 	                   SSKJLBW 	 ikun 			 0.763 	 0.505 	 0.080 	 https://learn2reg.grand-challenge.org/evaluation/f8ca0216-f5ac-4ad4-8899-c08bf642cf49/
3 	               wiebkeheyer 	 Challenge Organizers 			 0.748 	 0.504 	 0.063 	 https://learn2reg.grand-challenge.org/evaluation/f8ca0216-f5ac-4ad4-8899-c08bf642cf49/
4 	                        xi 	   			 0.666 	 0.487 	 0.086 	 https://learn2reg.grand-challenge.org/evaluation/f8ca0216-f5ac-4ad4-8899-c08bf642cf49/
5 	             xiangchen2O19 	 DA 			 0.445 	 0.276 	 0.000 	 https://learn2reg.grand-challenge.org/evaluation/f8ca0216-f5ac-4ad4-8899-c08bf642cf49/
6 	     pablomiguelgarcia199O 	   			 0.415 	 0.263 	 0.000 	 https://learn2reg.grand-challenge.org/evaluation/f8ca0216-f5ac-4ad4-8899-c08bf642cf49/
7 	                   INITIAL 	   			 0.180 	 0.276 	 5.000 	 https://learn2reg.gra

In [12]:
##pretty print with pandas
import pandas as pd
##printoptions precision
pd.set_option('precision', 3)

df = pd.DataFrame(columns=['#rank', 'user', 'team', '#score', 'DSC(mean)', 'LogJacDetStd(mean)'])
for ii,idx in enumerate(final_idx):
    team = users[idx].replace("0","O").zfill(25).replace("0", " ")
    gc_team = teams[idx] if str(teams[idx]) != 'None' else ' '
    df.loc[ii] = [ii+1, users[idx], gc_team, all_rank3[idx], DSC_mean[idx], LogJacDetStd_mean[idx]]
print(df)

  #rank                   user                  team  #score  DSC(mean)  \
0     1          alisonbrielee                         0.801      0.537   
1     2                SSKJLBW                  ikun   0.763      0.505   
2     3            wiebkeheyer  Challenge Organizers   0.748      0.504   
3     4                     xi                         0.666      0.487   
4     5          xiangchen2019                    DA   0.445      0.276   
5     6  pablomiguelgarcia1990                         0.415      0.263   
6     7                INITIAL                         0.180      0.276   

   LogJacDetStd(mean)  
0           3.825e-02  
1           8.008e-02  
2           6.330e-02  
3           8.637e-02  
4           1.754e-14  
5           1.754e-14  
6           5.000e+00  
